In [130]:
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine
from scipy import stats
import math as m

In [2]:
driver = 'mysql+pymysql:'
user = 'root'
password = '19693903'
ip = '127.0.0.1'
database = 'project_4'

In [3]:
connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

In [143]:
query = """
SELECT *
FROM flights
"""
query2 = """
SELECT * FROM airports
"""

In [144]:
data= pd.read_sql(query, engine)
airports_info = pd.read_sql(query2, engine)
data = data.merge(airports_info, left_on = 'depart_airpt', right_on='iata_code')

In [145]:
data = data.drop(columns = ('iata_code'), axis = 1).rename(columns = {'country_code':'depart_country'})
data = data.astype({'depart_delay': int,'arri_delay':int})
data['Total_delay'] = data['depart_delay'] + data['arri_delay']

In [146]:
q1, q3= np.percentile(data['Total_delay'],[25,75])
iqr = q3 - q1
upper_bound = q3 +(3.5 * iqr)
upper_bound

149.5

In [147]:
data = data[data['Total_delay'] <= upper_bound]

## Checking delay status:
By law, if a flight's departure delay time is longer than 15 min, it should be classify as **delayed**

In [148]:
data['Delay_status'] = np.where(data.depart_delay >= 15, 1,0)

### Classiying datetime into morning, afternoon and night

In [149]:
def classify_day(x):
    if x.hour <=12.0:
        return 0
    elif (x.hour > 12.0) and (x.hour <= 18.0):
        return 1
    else:
        return 2

In [150]:
data['day_range'] = data['schd_depart_time'].apply(classify_day)

### Europe summary statistics

In [151]:
eu_delays = data[['depart_delay', 'arri_delay','Total_delay']]
eu_delays =  eu_delays[np.logical_and(eu_delays['depart_delay']>0, eu_delays['arri_delay']>0)]
eu_delays.describe()

,depart_delay,arri_delay,Total_delay
count,36235.000000,36235.000000,36235.000000
mean,21.264164,19.067642,40.331806
std,16.406271,16.545273,31.711323
min,1.000000,1.000000,2.000000
25%,9.000000,7.000000,16.500000
50%,17.000000,14.000000,31.000000
75%,29.000000,27.000000,55.000000
max,86.000000,133.000000,149.000000


In [152]:
eu_dep_delay_mean = eu_delays.depart_delay.mean()
eu_dep_delay_std = eu_delays.depart_delay.std()
eu_dep_delay_var = eu_delays.depart_delay.var()
eu_t_delay_mean = eu_delays.Total_delay.mean()
eu_t_delay_std = eu_delays.Total_delay.std()
eu_t_delay_var = eu_delays.Total_delay.var()

europe_delay_mean = 12.0
#https://www.eurocontrol.int/sites/default/files/2019-07/eurocontrol_top_stats_-_june_2019.pdf

### Europe summary statistics without Spain

In [168]:
data_eu2 = data[data['depart_country'] != 'ES']

In [169]:
eu2_delays =  data_eu2[np.logical_and(data_eu2['depart_delay']>0, data_eu2['arri_delay']>0)]

In [170]:
eu2_delays.describe()

,depart_delay,arri_delay,Total_delay,Delay_status,day_range
count,28788.000000,28788.000000,28788.000000,28788.000000,28788.000000
mean,21.369251,19.154926,40.524177,0.579234,0.417813
std,16.082663,16.469234,31.370494,0.493691,0.536523
min,1.000000,1.000000,2.000000,0.000000,0.000000
25%,9.000000,7.000000,17.000000,0.000000,0.000000
50%,17.000000,14.000000,31.000000,1.000000,0.000000
75%,29.000000,27.000000,55.000000,1.000000,1.000000
max,85.000000,133.000000,149.000000,1.000000,2.000000


In [171]:
eu2_dep_delay_mean = eu2_delays.depart_delay.mean()
eu2_dep_delay_std = eu2_delays.depart_delay.std()
eu2_dep_delay_var = eu2_delays.depart_delay.var()
eu2_t_delay_mean = eu2_delays.Total_delay.mean()
eu2_t_delay_std = eu2_delays.Total_delay.std()
eu2_t_delay_var = eu2_delays.Total_delay.var()

## Spain summary statistics


In [180]:
data_es = data[data['depart_country'] == 'ES']
es_delays =  data_es[np.logical_and(data_es['depart_delay']>0, data_es['arri_delay']>0)]

In [154]:
es_delays.describe()

,depart_delay,arri_delay,Total_delay,Delay_status,day_range
count,7447.000000,7447.000000,7447.000000,7447.000000,7447.000000
mean,20.857929,18.730227,39.588156,0.533235,0.450651
std,17.596622,16.832855,32.987398,0.498928,0.548190
min,1.000000,1.000000,2.000000,0.000000,0.000000
25%,8.000000,6.000000,15.000000,0.000000,0.000000
50%,15.000000,13.000000,29.000000,1.000000,0.000000
75%,29.000000,25.000000,54.000000,1.000000,1.000000
max,86.000000,90.000000,149.000000,1.000000,2.000000


In [155]:
es_dep_delay_mean = es_delays.depart_delay.mean()
es_dep_delay_std = es_delays.depart_delay.std()
es_dep_delay_var = es_delays.depart_delay.var()
es_t_delay_mean = es_delays.Total_delay.mean()
es_tl_delay_std = es_delays.Total_delay.std()
es_t_delay_var = es_delays.Total_delay.var()

### Spain summary statistics without Barcelona


In [184]:
data_es_2 = data_es[data_es['depart_airpt'] != 'BCN']
es2_delays =  data_es_2[np.logical_and(data_es_2['depart_delay']>0, data_es_2['arri_delay']>0)]

In [185]:
es2_delays.describe()

,depart_delay,arri_delay,Total_delay,Delay_status,day_range
count,5648.000000,5648.000000,5648.000000,5648.000000,5648.000000
mean,20.434844,18.504958,38.939802,0.518591,0.470786
std,17.867525,17.238840,33.700941,0.499699,0.551762
min,1.000000,1.000000,2.000000,0.000000,0.000000
25%,7.000000,6.000000,14.000000,0.000000,0.000000
50%,15.000000,13.000000,28.000000,1.000000,0.000000
75%,28.000000,25.000000,52.000000,1.000000,1.000000
max,86.000000,90.000000,149.000000,1.000000,2.000000


In [186]:
es2_dep_delay_mean = es2_delays.depart_delay.mean()
es2_dep_delay_std = es2_delays.depart_delay.std()
es2_dep_delay_var = es2_delays.depart_delay.var()
es2_t_delay_mean = es2_delays.Total_delay.mean()
es2_tl_delay_std = es2_delays.Total_delay.std()
es2_t_delay_var = es2_delays.Total_delay.var()

### Spain summary statistics of most important airports (BCN, MAD, MAL, BMI)


In [193]:
spain_airpts_main = ['BCN', 'MAD', 'PMI', 'LPA']

In [197]:
data_es_3 = data_es[data_es['depart_airpt'].isin(spain_airpts_main)]
es3_delays =  data_es_3[np.logical_and(data_es_3['depart_delay']>0, data_es_3['arri_delay']>0)]

In [198]:
es3_delays.describe()

,depart_delay,arri_delay,Total_delay,Delay_status,day_range
count,4728.000000,4728.000000,4728.000000,4728.000000,4728.000000
mean,20.736041,18.976946,39.712986,0.524746,0.478638
std,17.647925,16.711925,32.877524,0.499440,0.561029
min,1.000000,1.000000,2.000000,0.000000,0.000000
25%,8.000000,7.000000,16.000000,0.000000,0.000000
50%,15.000000,14.000000,28.000000,1.000000,0.000000
75%,30.000000,25.250000,54.250000,1.000000,1.000000
max,86.000000,90.000000,149.000000,1.000000,2.000000


In [199]:
es3_dep_delay_mean = es3_delays.depart_delay.mean()
es3_dep_delay_std = es3_delays.depart_delay.std()
es3_dep_delay_var = es3_delays.depart_delay.var()
es3_t_delay_mean = es3_delays.Total_delay.mean()
es3_tl_delay_std = es3_delays.Total_delay.std()
es3_t_delay_var = es3_delays.Total_delay.var()

### Barcelona summary statistics


In [156]:
data_bcn = data[data['depart_airpt'] == 'BCN']
bnc_delays =  data_bcn[np.logical_and(data_bcn['depart_delay']>0, data_bcn['arri_delay']>0)]

In [157]:
bnc_delays.describe()

,depart_delay,arri_delay,Total_delay,Delay_status,day_range
count,1799.000000,1799.000000,1799.000000,1799.000000,1799.000000
mean,22.186215,19.437465,41.623680,0.579211,0.387437
std,16.652681,15.472512,30.559068,0.493823,0.532041
min,1.000000,1.000000,3.000000,0.000000,0.000000
25%,9.000000,7.000000,17.000000,0.000000,0.000000
50%,18.000000,16.000000,33.000000,1.000000,0.000000
75%,32.000000,27.000000,58.000000,1.000000,1.000000
max,86.000000,74.000000,147.000000,1.000000,2.000000


In [158]:
bnc_dep_delay_mean = bnc_delays.depart_delay.mean()
bnc_dep_delay_std = bnc_delays.depart_delay.std()
bnc_dep_delay_var = bnc_delays.depart_delay.var()
bnc_t_delay_mean = bnc_delays.Total_delay.mean()
bnc_t_delay_std = bnc_delays.Total_delay.std()
bnc_t_delay_var = bnc_delays.Total_delay.var()

### Madrid summary statistics


In [159]:
data_mad = data[data['depart_airpt'] == 'MAD']
mad_delays =  data_mad[np.logical_and(data_mad['depart_delay']>0, data_mad['arri_delay']>0)]

In [160]:
mad_dep_delay_mean = mad_delays.depart_delay.mean()
mad_dep_delay_std = mad_delays.depart_delay.std()
mad_dep_delay_var = mad_delays.depart_delay.var()
mad_t_delay_mean = mad_delays.Total_delay.mean()
mad_t_delay_std = mad_delays.Total_delay.std()
mad_t_delay_var = mad_delays.Total_delay.var()

### Vueling summary statistics


In [161]:
data_vueling = data[data['airln_name'] == 'Vueling']
vueling_delays =  data_vueling[np.logical_and(data_vueling['depart_delay']>0, data_vueling['arri_delay']>0)]

In [162]:
vueling_delays.describe()

,depart_delay,arri_delay,Total_delay,Delay_status,day_range
count,857.000000,857.000000,857.000000,857.000000,857.000000
mean,22.914819,19.554259,42.469078,0.584597,0.436406
std,17.858373,17.259622,33.842361,0.493079,0.545569
min,1.000000,1.000000,2.000000,0.000000,0.000000
25%,9.000000,6.000000,17.000000,0.000000,0.000000
50%,18.000000,15.000000,33.000000,1.000000,0.000000
75%,32.000000,28.000000,58.000000,1.000000,1.000000
max,86.000000,86.000000,148.000000,1.000000,2.000000


In [190]:
vue_dep_delay_mean = vueling_delays.depart_delay.mean()
vue_dep_delay_std = vueling_delays.depart_delay.std()
vue_dep_delay_var = vueling_delays.depart_delay.var()
vue_t_delay_mean = vueling_delays.Total_delay.mean()
vue_t_delay_std = vueling_delays.Total_delay.std()
vue_t_delay_var = vueling_delays.Total_delay.var()

# Hypothesis testing

## 1- Its is more likely to have delay time in Spain airports than in the rest of europe. 

H0: The mean departure time in minutes in Spain is equal or lower than in the rest of Europe


H1: The mean departure time in minutes in Spain is greater than in the rest of Europe

#### Testing with the european departure delay mean by Eurocontrol

In [163]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.
alpha = 0.05

result_ttest = stats.ttest_1samp(es_delays['depart_delay'], europe_delay_mean)

if (result_ttest[0] > 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis')

print(result_ttest)

Reject null hypothesis, we have evidence for alternative hypothesis

Ttest_1sampResult(statistic=43.44037678680821, pvalue=0.0)


#### Testing with the european departure delay mean of my sample

In [164]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.
alpha = 0.05

result_ttest = stats.ttest_1samp(es_delays['depart_delay'], eu_dep_delay_mean)

if (result_ttest[0] > 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis\n')

print(result_ttest)

We dont have statistical evidence to reject the null hypothesis

Ttest_1sampResult(statistic=-1.9922270429197275, pvalue=0.04638260577133076)


#### Testing with the european departure delay mean of my sample without Spain

In [172]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.
alpha = 0.05

result_ttest = stats.ttest_1samp(es_delays['depart_delay'], eu2_dep_delay_mean)

if (result_ttest[0] > 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis\n')

print(result_ttest)

We dont have statistical evidence to reject the null hypothesis

Ttest_1sampResult(statistic=-2.5075846498609233, pvalue=0.012177058994911234)


## 2-The Barcelona airport is the airport with less delays time in average of Spain. 

H0: The mean departure time in minutes in BCN is equal or greater than in the rest of Spain.


H1: The mean departure time in minutes in BCN is lower than in the rest of Spain.

In [187]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.

alpha = 0.05

result_ttest = stats.ttest_1samp(bnc_delays['depart_delay'], es_dep_delay_mean)

if (result_ttest[0] < 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis\n')

print(result_ttest)

We dont have statistical evidence to reject the null hypothesis

Ttest_1sampResult(statistic=3.3831615661462213, pvalue=0.0007319411064950384)


#### Comparing Barcelona with Spain without Barcelona data

In [188]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.

alpha = 0.05

result_ttest = stats.ttest_1samp(bnc_delays['depart_delay'], es2_dep_delay_mean)

if (result_ttest[0] < 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis\n')

print(result_ttest)

We dont have statistical evidence to reject the null hypothesis

Ttest_1sampResult(statistic=4.460765613153488, pvalue=8.672222381230959e-06)


#### Comparing Barcelona with Spain most important airports in size

In [200]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.

alpha = 0.05

result_ttest = stats.ttest_1samp(bnc_delays['depart_delay'], es3_dep_delay_mean)

if (result_ttest[0] < 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis\n')

print(result_ttest)

We dont have statistical evidence to reject the null hypothesis

Ttest_1sampResult(statistic=3.6936139930406617, pvalue=0.0002276955826635293)


## 3- Vueling is the air carrier with more delays on average

H0: The mean departure time in minutes in Vueling is equal or lower than in the rest of airlines.


H1: The mean departure time in minutes in Vueling is greater than in the rest of airlines.

In [191]:
#With a condifence level of 5%, lets test to see if we can reject our null hypotehsis.

alpha = 0.05

result_ttest = stats.ttest_1samp(vueling_delays['depart_delay'], eu_dep_delay_mean)

if (result_ttest[0] > 0) & (result_ttest[1] < alpha):
    print (f"Reject null hypothesis, we have evidence for alternative hypothesis\n")
else:
    print('We dont have statistical evidence to reject the null hypothesis\n')

print(result_ttest)

Reject null hypothesis, we have evidence for alternative hypothesis

Ttest_1sampResult(statistic=2.705856435208683, pvalue=0.006948359506429609)
